In [18]:
import cv2
import numpy as np
import os

In [19]:
OUTPUT_FOLDER = "output_images"      #Folder where output images will bes aved.

AGE_PRIORITY = {

    "Star": 3,                #Child
    "Triangle": 2,            #Elderly
    "Square": 1               #Adult
}

EMERGENCY_PRIORITY = {
    "Red": 3,                 #Severe
    "Yellow": 2,              #Mild
    "Green": 1               #Safe
}

CAMP_CAPACITY = {
    "Blue": 4,
    "Pink": 3,
    "Grey": 2
}


In [20]:
def setup_output_folder():
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)

#Creates a folder by the name assigned to var OUTPUT_FOLDER in case it isnt already present.

In [21]:
def distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

#Calculates Euclidean distance between centroids.

In [22]:
#    Detects the geometric shape of a contour using contour approximation.

def detect_shape(cnt):
    peri = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.03 * peri, True)
    v = len(approx)

    if v == 3:
        return "Triangle"
    elif v == 4:
        return "Square"
    elif v >= 10:
        return "Star"
    elif v >= 6:
        return "Circle"
    return "Unknown"

In [23]:
#    Determines the dominant color inside a contour using HSV color space.

def detect_color(cnt, hsv):
    mask = np.zeros(hsv.shape[:2], dtype="uint8")
    cv2.drawContours(mask, [cnt], -1, 255, -1)
    mask = cv2.erode(mask, np.ones((5,5), np.uint8), 1)

    h, s, v, _ = cv2.mean(hsv, mask=mask)

    if s < 40 and v > 180:
        return "Grey"

    if h < 10 or h > 160:
        return "Red"
    elif 18 < h < 38:
        return "Yellow"
    elif 38 < h < 85:
        return "Green"
    elif 90 < h < 125:
        return "Blue"
    elif 125 < h < 155:
        return "Pink"

    return "Unknown"


In [24]:
def load_image():
    file_name = input("Enter image name (e.g. 2.png): ").strip()     #Accepts user input for image
    input_path = os.path.join("images", file_name)

    image = cv2.imread(input_path)                                 #Validates if entered image is actually present in the input fold er.
    if image is None:
        print("Image not found")
        exit()

    return image, file_name, input_path

In [25]:
    # Prepares the image for contour detection by:
    # - Resizing
    # - Masking land regions
    # - Thresholding
    # - Extracting contours


def preprocess_image(image):
    image = cv2.resize(image, (800, 500))
    display = image.copy()
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    land_lower = np.array([43,110,91])
    land_upper = np.array([77,238,161])
    land_mask = cv2.inRange(hsv, land_lower, land_upper)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7,7), 2)
    _, thresh = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)

    thresh[land_mask == 255] = 0
    display[land_mask == 255] = (0,200,255)

    contours, _ = cv2.findContours(
        thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
    )

    return display, hsv, contours


In [26]:
"""
    Identifies people and camps from contours
    based on shape and color.

    Returns:
        people= list of detected people
        camps =list of detected camps

"""


def extract_people_and_camps(contours, hsv, display):
    people = []
    camps = []

    for cnt in contours:
        if cv2.contourArea(cnt) < 200:
            continue                               #To ignore extra smaller, unwanted contours

        shape = detect_shape(cnt)
        color = detect_color(cnt, hsv)

        x, y, w, h = cv2.boundingRect(cnt)
        cx, cy = x + w // 2, y + h // 2

        cv2.rectangle(display, (x,y), (x+w,y+h), (0,255,255), 1)
        cv2.putText(
            display, f"{color} {shape}", (x,y-6),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,0,255), 1
        )

    #Person detection
        if shape in AGE_PRIORITY and color in EMERGENCY_PRIORITY:
            people.append({
                "center": (cx,cy),
                "age_p": AGE_PRIORITY[shape],
                "emg_p": EMERGENCY_PRIORITY[color],
                "base_priority": AGE_PRIORITY[shape] * EMERGENCY_PRIORITY[color]
            })


    #Camp Detection
        if shape == "Circle" and color in CAMP_CAPACITY:
            camps.append({
                "center": (cx,cy),
                "color": color,
                "capacity": CAMP_CAPACITY[color],
                "assigned": []
            })

    return people, camps


In [27]:
#Assigns people to camp based on priority score and later sort the people assigned in particular camps.


def assign_people_to_camps(people, camps):
    assignments = []

    for p in people:
        for c in camps:
            d = distance(p["center"], c["center"])
            score = p["base_priority"] / (d + 1)
            assignments.append({"p": p, "c": c, "score": score})

    assignments.sort(key=lambda x: x["score"], reverse=True)     # Higher Score is prefereed for given camp.

    used = set()
    for a in assignments:
        if id(a["p"]) in used:
            continue
        if len(a["c"]["assigned"]) >= a["c"]["capacity"]:
            continue
        a["c"]["assigned"].append(a["p"])
        used.add(id(a["p"]))


In [28]:
""" -Prints the required output for enetered image:
    - Camp-wise assignments
    - Priority scores
"""

def print_results(file_name, people, camps):
    print(f"IMAGE: {file_name}\n")

    camp_scores = []
    for c in camps:
        print(f"{c['color']} camp:")
        for p in c["assigned"]:
            print([p["age_p"], p["emg_p"]])
        camp_scores.append(sum(p["base_priority"] for p in c["assigned"]))

    ratio = sum(camp_scores) / len(people) if people else 0
    print("Camp priority scores:", camp_scores)
    print("Image Priority Ratio:", round(ratio, 3))


In [29]:
def save_output(display, file_name):                     #Save the annotated image into assigned output folder
    output_path = os.path.join(OUTPUT_FOLDER, file_name)
    cv2.imwrite(output_path, display)
    print(f"\n Output saved to: {output_path}")

In [30]:
#Main program with proper sequencing and control-flow.

def main():
    setup_output_folder()

    image, file_name, _ = load_image()
    display, hsv, contours = preprocess_image(image)

    people, camps = extract_people_and_camps(contours, hsv, display)
    assign_people_to_camps(people, camps)

    print_results(file_name, people, camps)
    save_output(display, file_name)

In [31]:
if __name__ == "__main__":
    main()                   #calls main function and subsequently other in set sequence.


IMAGE: 3.png

Grey camp:
[3, 3]
[2, 2]
Blue camp:
[1, 3]
[1, 1]
[1, 2]
[1, 1]
Pink camp:
[3, 2]
[2, 2]
[3, 1]
Camp priority scores: [13, 7, 13]
Image Priority Ratio: 3.667

 Output saved to: output_images\3.png
